In [128]:
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd

In [4]:
url = "http://www.veterans.gc.ca/eng/contact/map"
resp = requests.get(url)
text = resp.text

In [10]:
soup = BeautifulSoup(text, 'html.parser')

In [133]:
def get_data_from_details(details):
    office_name_w_whitespace = details.summary.text
    office_name = office_name_w_whitespace.strip()
    p = details.find("p")
    address = ', '.join([x for x in p.contents if type(x) != bs4.element.Tag])    
    a = details.find("a")    
    latLngStr = a.attrs['href'].split('(')[-1].split(')')[0].split(',')[:2]   
    lat = float(latLngStr[0]) 
    lng = float(latLngStr[1])
    return {
        'office_name': office_name,
        'address': address,
        'lat': lat,
        'lng': lng
           }

In [134]:
office_data = []
for details in soup.find_all('details'):
    try:
        office_data.append(get_data_from_details(details))
    except:
        print('exception!')

In [135]:
office_data_2 = [x for x in office_data if 'office' in x['office_name'].lower()]

In [136]:
len(office_data_2)

38

In [137]:
df = pd.DataFrame.from_dict(office_data_2)

In [82]:
df.to_csv("vac_office_data.csv", index=False)